In [6]:
from submissions.ab_submission.dataLoaders import imageDataLoader
from submissions.ab_submission.object_detector import ObjectDetector
from submissions.ab_submission import dataset

import importlib

importlib.reload(dataset)

<module 'submissions.ab_submission.dataset' from '/home/ali/git/follicles_detection/submissions/ab_submission/dataset.py'>

# Model evaluation

In [7]:
import glob
import pandas as pd

In [8]:
train_files = glob.glob("./data/train/*.jpg")
test_files = glob.glob("./data/test/*.jpg")
train_label = pd.read_csv("./data/train/labels.csv")
test_label = pd.read_csv("./data/test/labels.csv")

In [9]:
model = ObjectDetector(ramp_mode=False)
model.load(boxPixelClassifier="./params/boxPixelClassifier_opencv.joblib", follicleClassifier="./params/follicleClassifier2.model")
model.fit(train_files, train_label)
#model.save(boxPixelClassifier="./params/boxPixelClassifier_opencv.joblib",follicleClassifier="./params/follicleClassifier2.model")

Model loaded from ./params/follicleClassifier2.model
Fitting boxPixelClassifier
Fitting follicleClassifier


In [5]:
y_hat = model.predict(test_files)

In [10]:
import numpy as np
import glob
import os
import pandas as pd
from matplotlib import pyplot as plt

from PIL import Image, ImageDraw, ImageFont

font_size = 60
if os.name != 'nt':
    font = ImageFont.truetype("/usr/share/fonts/truetype/freefont/FreeMono.ttf", font_size)
else:
    font = ImageFont.truetype("C:/Windows/Fonts/Arial/arialbd.ttf", font_size)

def write_rectangle(image, preds, folder=None, filename=None):
    img = Image.fromarray(image)
    img_draw = ImageDraw.Draw(img)
    for pred in preds:
        x1, y1, x2, y2 = pred["bbox"]
        label = pred["class"]
        img_draw.rounded_rectangle(((x1, y1), (x2,y2)), fill=None, outline="black", width=5)
        img_draw.text((x1, y1-70), label, font=font, fill="black")

    if folder is not None and filename is not None:
        img.save(f"./data/{folder}/{filename}")
    
    return np.array(img)

In [11]:
test_loader = imageDataLoader(test_files, test_label[["filename","xmin","xmax","ymin","ymax","label"]])
train_loader = imageDataLoader(train_files, train_label[["filename","xmin","xmax","ymin","ymax","label"]])
predict_loader = imageDataLoader(test_files, None)

In [12]:
for data, preds, filename in zip(test_loader.get_samples(), y_hat, test_loader.X_filenames):
    write_rectangle(data[0], preds, "test_predicted", filename)

NameError: name 'y_hat' is not defined

In [13]:
reload_data = False

train_dataset = dataset.folliclesDataset(
    train_loader,
    data_augmentation=False,
    local_path="./datasets/train",
    box_classifier=model._get_box_list,
    verbose=False,
    order="box_ratio",
    force_reload=reload_data,
    mode="all"
)

test_dataset = dataset.folliclesDataset(
    test_loader,
    data_augmentation=False,
    local_path="./datasets/test",
    box_classifier=model._get_box_list,
    verbose=False,
    order="box_ratio",
    force_reload=reload_data,
    mode="crop"
)

In [31]:
for file in train_loader.X_filenames:
    data = train_dataset[file]
    image = train_loader.get_sample(file)[0]
    preds = [x[1] for x in data]
    for i in range(len(preds)):
        preds[i]["class"] = model._class_dict[int(preds[i]["label"])]
    write_rectangle(image, preds, "train_box_for_cnn", file)